In [1]:
from directory_area_verde_load import definir_diretorio 
project_path = definir_diretorio()

import os

from utils import (
    get_data_diretorio,
    check_crs,
    save_parquet_excel
)
from utils.downloads import (
    download_malha_geosampa
)

# Quadra Viária (praças)

In [2]:
data_path = get_data_diretorio(project_path)

Deu problema ao baixar o arquivo, o que deveria ser salvo como uma pasta compactada, foi baixado como um outro tipo de arquivo. Preciso corrigir isso na função depois. 

 f'''{os.path.join(
                    'vsizip',
                    file_apth
                )}'''

# Importar dados 

In [3]:
gdf_quadra= download_malha_geosampa(
    'quadra_viaria_editada',
    data_path    
)

OK, sem a paginação deu certo, vamos tentar com a paginação agora.

Com a paginação está dando errado. Vou confirmar com o Henrique depois qula o motivo disso, já que ele entende melhor o código da paginação melhor que eu.

In [4]:
gdf_quadra.shape

(30000, 8)

In [5]:
gdf_quadra.sample(2)

,cd_identif,cd_identi0,tx_tipo_qu,tx_escala,tx_ano_ref,qt_area_me,sg_fonte_o,geometry
2309,2149,3601,Praca_Canteiro,1:1.000,2017,66,SMUL/GEOINFO,"POLYGON ((330978.413 7381803.928, 330993.498 7..."
734,734,934,Praca_Canteiro,1:1.000,2004,3232,SMUL/GEOINFO,"POLYGON ((343205.586 7391059.388, 343205.796 7..."


## Padronizaçã de colunas

In [6]:
drop_quadra_cols={
    'tx_escala',
    'tx_ano_ref'
}

gdf_quadra.drop(
    columns=drop_quadra_cols,
    axis=1,
    inplace=True
)

In [7]:
rename_quadra_cols = {
    'cd_identif' : 'cd_quadra'
}

gdf_quadra.rename(
    rename_quadra_cols,
    axis=1,
    inplace=True
)

## Conferir CRS

In [8]:
gdf_quadra = check_crs(gdf_quadra)

# Filtrar por praças

In [9]:
gdf_quadra.columns

Index(['cd_quadra', 'cd_identi0', 'tx_tipo_qu', 'qt_area_me', 'sg_fonte_o',
       'geometry'],
      dtype='object')

In [10]:
gdf_quadra['tx_tipo_qu'].unique()

array(['Quadra', 'Praca_Canteiro', 'CET', 'Borda'], dtype=object)

Eu fui orientada a pegar apenas o Praca_Canteiro, mas caso seja interessante, é possível pegar outros tipos também.

In [11]:
praca = gdf_quadra['tx_tipo_qu'].unique()[1]

Por algum motivo, quando eu digitava 'Praca_Canteiro', o loc não reconhecia, por isso passei como uma variável, para pegar o nome certinho.

In [12]:
gdf_quadra = gdf_quadra.loc[gdf_quadra['tx_tipo_qu']==praca]

In [13]:
gdf_quadra.shape

(10926, 6)

Acredito que após a paginação, o número de praças será maior.

In [14]:
gdf_quadra.sample(2)

,cd_quadra,cd_identi0,tx_tipo_qu,qt_area_me,sg_fonte_o,geometry
4280,4005,5853,Praca_Canteiro,78,SMUL/GEOINFO,"POLYGON ((340826.318 7394079.827, 340826.263 7..."
8333,8151,11256,Praca_Canteiro,1137,SMUL/GEOINFO,"POLYGON ((332799.844 7393868.552, 332806.386 7..."


# Salvar GDF

In [15]:
save_parquet_excel(
    gdf_quadra, 
    fname= 'quadra_viaria_editada',
    data_path = data_path,
    data_subpath = 'assets'
)